<a href="https://colab.research.google.com/github/Ptisni/Cartpole-course/blob/main/Cartpole_PID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3 Ways to Control a Cartpole**
### Session 1 - Introduction and PID Control
By Peter Tisnikar \\
February 2022 (Version 1)  

---  
  

**Welcome** to this tutorial session on control! Hopefully you've found the introductory lecture useful. In this notebook you can find the environment in which we will control a cartpole using a PID controller which we will build from scratch and then tune, so the cartpole will remain upright for the duration of the simulation.

#### Before you get started, a brief introduction to Jupyter notebooks (which you can skip if you have used them before and know how they work):  

Jupyter notebooks are interactive notebooks built from **cells**, which can contain either text (like this one!), or snippets of code. You can run each cell individually by pressing the run button in the upper left corner of each cell, or you can select the **Run All**  (Ctrl/Cmd + F9) from the **Runtime** menu above.  

In this document, the text cells will guide you through building your own implementation and tuning of the PID controller, along with minimal theory. They will also contain all instructions for the code cells directly below them and tell you if you need to modify them. Some of the code cells are helper functions (e.g. to visualise the environment or install the packages), so you will need to run and **not change** those.  

To start, run the below two cells to install the environment we will use ([Open AI Gym](https://gym.openai.com/envs/CartPole-v1/)), and to enable the rendering of the simulation in Google Colab. Once you are done with that, you can proceed to the next text cell to start building the PID controller.

*Code in cells 1 and 2 reused from [here](https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb)*. 


In [ ]:
# DO NOT MODIFY THIS CELL!
#--------------------------------------------------------------
!apt-get update > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[all] > /dev/null 2>&1

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# DO NOT MODIFY THIS CELL!
#--------------------------------------------------------------
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
import numpy as np
import time

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

## **Building a PID controller**  

Now we can start building the PID controller! Observe the sketch below. It has an environment, which runs in a simulation loop which passes a force through and then receives some observations about the environment. The episode also terminates if the cart moves outside of a certain threshold (x position less than 2.4 m and 24 degrees offset of the pole). Line 11 ensures that we conform with the way the environment handles inputs: The force can either be 1 unit pushing the cart to the right, or 1 unit pushing the cart to the left.

**Challenge**: Using the supplementary lecture slides and the procedure below, construct a PID controller and tune it so the simulation episode lasts for 10 seconds! You can check how long the episode lasts by checking out the duration of the video.  

---


  
    
      
### Procedure

Firstly, we need to break down the observations we get from the environment (what are the values we are actually getting?). Looking into the Open AI Gym cartpole documentation, we can see that observation[0] and observation[1] relate to the position and velocity of the **cart**, while observation[2] and observation[3] are the angle and angular velocity of the **pole** respectively. An important thing to consider is what it means that the angle of the pole is 0: This simply means that the pole is in upright position. **This is what we want**. Because of this, any offset of the pole from the upright position is the **error** we are trying to minimise!  

Okay, so how do we actually start building a PID controller?   

Well, we first need to initialise our three gains: the proportional gain, then the derivative gain, and lastly the integral gain. They can be arbitrary for now, but it is best to set them as 1 while we are writing the code. **It is important to consider** where we are going to put them: they can be outside of the simulation loop so we don't re-define them every time, as they are fixed!  

Next, we want to **initialise our integral error**. Be careful about placement of this definition, as we want to keep accumulating the error across the simulation steps, so it needs to be outside of the loop!  

Lastly, we need to start calculating the **command** variable. Recall, in a PID controller, the command is just the equation below:  

<center> $u(t) = K_{p}*\theta(t) + K_{d}*\dot{\theta}(t) + K_{i}*∫θ(t)dt$ </center>

Where $\theta$ represents our error (pole angle). However, this equation describes the controller in **continuous time**, hence the derivative in the second term, and an integral in the third. In this exercise, we are dealing with a **discrete time** controller which allows us to turn derivatives into differences, and integrals into sums, simplifying the equation to the following:
<center> $u_{t} = K_{p} * \theta_{t} + K_{d} * (\theta_{t} - \theta_{t-1}) + K_{i} * (\theta_{t-1} + \theta_{t})$ </center>

Here, t denotes the current time step, and t-1 signifies the previous time step.

Okay, now we have all the necessary steps to build a PID controller! As one last hint in this section, I would like you to think about what the derivative term actually means - what does a difference between 2 angles (over time) actually represent? Does this make implementation even easier? Think about it and try to implement the controller below!



In [ ]:
env = gym.make("CartPole-v1")
env = wrap_env(env)

observation = env.reset()
force = 0

for _ in range(50000): # This is the simulation loop
  env.render()

  observation, reward, done, info = env.step(force)

  #TODO: Write PID controller!

  force = 1 if command > 0 else 0 # TODO: Change variable name of 'error' to match the name of your PID calculation!
  if done:
    break

env.close()
show_video()

# **Solution and Bonus Questions**  
See below for a solution to this problem!
## Bonus Questions:
1) Using the materials, think about how you could write a tuner for the PID controller. Which method would you use?

  
  
2) Consider a more difficult version of this environment, where you receive observations at a **lower** frequency than the simulation rate (i.e., for every 5 simulation steps you receive 1 observation from the environment). What modifications to the controller would you make to ensure it can hold up the pendulum? You can have a go with the environment in the last cell of this notebook.
-  
-  
-  
-  
-  
-  
-  
-  
-  
- 
-  
-  
-  
-  
- 

In [ ]:
# SOLUTION
# DO NOT MODIFY THIS CELL!
#--------------------------------------------------------------
env = gym.make("CartPole-v1")
env = wrap_env(env)

observation = env.reset()

Kp = 120
Ki = 90
Kd = 45

force = 0
error_integral = 0
for _ in range(50000):

  env.render()
  observation, reward, done, info = env.step(force)
  angle_error = observation[2]
  angular_velocity = observation[3]

  error_integral = error_integral + angle_error

  command = Kp * (angle_error) + Kd * (angular_velocity) + Ki * (error_integral)

  force = 1 if command > 0 else 0
  if done:
    break

env.close()
show_video()

In [ ]:
# ENVIRONMENT FOR ADDITIONAL QUESTION 2
#--------------------------------------------------------------
env = gym.make("CartPole-v1")
env = wrap_env(env)

observation = env.reset()

force = 0

for i in range(50000):

  env.render()

  if i%4 == 0 and not done:
    observation, reward, done, info = env.step(force)
  elif not done: 
    done = env.step(force)[2]

# TODO: Implement PID Controller capable of operating only on limited observations. 
# Hint: You can decide how much force to apply based on error (e.g. in 4 time steps, you can decide to apply a force for 1, 2, 3, or all 4 timesteps)

  if done:
    break

env.close()
show_video()